# 1 Download Raw Data

This module is used for downloading the original Excel files that contain the historical LAM data.

Run all cells of this notebook to get the data of Askisto, Mäntsälä and Kemijärvi to "raw_dataset.csv".

In [5]:
import urllib.request
import calendar
import glob
import pandas as pd
import subprocess
from dateutil import parser

In [64]:
def get_file_url(y, m, file_prefix, next_month=False):
    base_url = "https://aineistot.liikennevirasto.fi/lam/reports/LAM/"
    dir_m = m
    dir_y = y
    if next_month:
        if dir_m < 12:
            dir_m += 1
        else:
            dir_m = 1
            dir_y += 1
    dir_name = "20{:02}{:02}11/".format(dir_y, dir_m)
    first_date = parser.parse("20{:02}-{:02}-01".format(y, m))
    d = calendar.monthrange(first_date.year, first_date.month)[1]
    file_name = file_prefix + "_20{:02}{:02}01_20{:02}{:02}{:02}.xls".format(y, m, y, m, d)
    file_url = base_url + dir_name + file_name
    return file_url, file_name

def download_xls_files(file_prefix, to_location="./data/"):
    for y in range(10, 18):
        for m in range(1, 13):
            try:
                file_url, file_name = get_file_url(y, m, file_prefix)
                urllib.request.urlretrieve(file_url, to_location + file_name)
                print("downloaded: " + file_name)
            except: 
                try:
                    file_url, file_name = get_file_url(y, m, file_prefix, next_month=True)
                    urllib.request.urlretrieve(file_url, to_location + file_name)
                    print("downloaded: " + file_name)
                except:
                    print("not found: " + file_url)
            
def download_all_excel_files(data_dir='./data/'):
    file_prefixes = ["168_Askisto", "110_M%c3%84NTS%c3%84L%c3%84", "1403_KEMIJ%c3%84RVI"]
    for prefix in file_prefixes:
        download_xls_files(file_prefix=prefix, to_location=data_dir)     
        
def convert_xls_to_csv(data_dir="./data/"):
    for f in (glob.glob("{}*.xls".format(data_dir))):
        # to run this process you need ssconvert: with mac OS run 'brew install gnumeric' first
        subprocess.call(["ssconvert", f, f[:-3] + "csv"])

def read_csv_files_to_dataframe(columns, data_dir="data/"):
    dfs = []
    for f in (glob.glob("{}*.csv".format(data_dir))):
        try:
            df = pd.read_csv(f)
            df.columns = columns 
            dfs.append(df)
        except:
            print("Failed: " + f)
    df = pd.concat(dfs)
    return df
        
def export_dataframe_to_csv(df, filename="raw_dataset.csv"):
    df.to_csv(filename, index=False)

In [7]:
# download all files
download_xls_files()

In [32]:
# convert xls files to csv files
convert_xls_to_csv()

In [59]:
# read the csv files to a DataFrame
columns = [
    'location_id', 
    'location_name', 
    'date', 
    'direction', 
    'vehicle_type',  
    'hour_1', 
    'hour_2',
    'hour_3',
    'hour_4',
    'hour_5', 
    'hour_6', 
    'hour_7',
    'hour_8',
    'hour_9',
    'hour_10',
    'hour_11',
    'hour_12',
    'hour_13',
    'hour_14',
    'hour_15',
    'hour_16',
    'hour_17',
    'hour_18',
    'hour_19',
    'hour_20',
    'hour_21',
    'hour_22',
    'hour_23',
    'hour_24',
]
data = read_csv_files_to_dataframe(columns).fillna(0)

Failed: data/110_M%c3%84NTS%c3%84L%c3%84_20140401_20140430.csv


In [63]:
# Inspect the DataFrame and ensure it looks OK
print(data.info())

data.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 107420 entries, 0 to 433
Data columns (total 29 columns):
location_id      107420 non-null int64
location_name    107420 non-null object
date             107420 non-null object
direction        107420 non-null int64
vehicle_type     107420 non-null object
hour_1           107420 non-null float64
hour_2           107420 non-null float64
hour_3           107420 non-null float64
hour_4           107420 non-null float64
hour_5           107420 non-null float64
hour_6           107420 non-null float64
hour_7           107420 non-null float64
hour_8           107420 non-null float64
hour_9           107420 non-null float64
hour_10          107420 non-null float64
hour_11          107420 non-null float64
hour_12          107420 non-null float64
hour_13          107420 non-null float64
hour_14          107420 non-null float64
hour_15          107420 non-null float64
hour_16          107420 non-null float64
hour_17          107420 non-null float

,location_id,location_name,date,direction,vehicle_type,hour_1,hour_2,hour_3,hour_4,hour_5,...,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,hour_24
0,168,Askisto,2016/01/01,1,11 HA-PA,183.0,211.0,146.0,121.0,108.0,...,742.0,774.0,696.0,599.0,502.0,399.0,305.0,240.0,178.0,93.0
1,168,Askisto,2016/01/01,1,12 KAIP,1.0,2.0,0.0,1.0,1.0,...,8.0,1.0,5.0,3.0,3.0,0.0,3.0,3.0,2.0,0.0
2,168,Askisto,2016/01/01,1,13 Linja-autot,4.0,4.0,5.0,2.0,4.0,...,3.0,3.0,3.0,3.0,1.0,3.0,2.0,1.0,7.0,2.0
3,168,Askisto,2016/01/01,1,14 KAPP,2.0,0.0,0.0,1.0,2.0,...,0.0,2.0,0.0,1.0,2.0,2.0,1.0,2.0,1.0,0.0
4,168,Askisto,2016/01/01,1,15 KATP,0.0,4.0,4.0,1.0,1.0,...,5.0,4.0,2.0,3.0,3.0,1.0,3.0,3.0,5.0,1.0


In [65]:
# export the DataFrame to a csv file
export_dataframe_to_csv(data)